# Metoda podziału i ograniczeń – algorytm Little’a dla zagadnienia komiwojażera

<b>Grupa: </b> 4a, czwartek 14:30 – 16:00 

<b>Data zajęć: </b> 27.05.2021

<b> Skład zespołu: </b>

-Zuzanna Zielińska 

-Zofia Lenarczyk 

-Maciej Kucharski 

-Łukasz Rams

### Importy potrzebnych bibliotek

In [1]:
import numpy as np
from typing import Tuple, Optional, List

## Krok 1 - Redukcja macierzy i wyznaczenie najmniejszego kosztu redukcji

In [2]:
def reduce_matrix(matrix: np.ndarray) -> Optional[Tuple[np.ndarray, int]]:
    """
    Funkcja realizuje redukcję macierzy kwadratowej.
    W każdym wierszu wyszukiwany jest element najmniejszy, który następnie jest dodawany do kosztu redukcji oraz
    odejmowany (redukowany) od wszystkich elementów w wierszu.
    Analogiczne działanie dla kolumn macierzy.

    :param matrix: macierz wejściowa
    :return: zredukowana macierz, koszt redukcji
    """

    # Sprawdzenie czy macierz jest kwadratowa:
    if matrix.shape[0] != matrix.shape[1]:
        print('Niepoprawne wymiary macierzy wejściowej')
        return None
    
    print("Krok 1\n")

    reduction_cost: int = 0  # Koszt redukcji
    reduced_rows_matrix: np.ndarray = np.zeros(matrix.shape)  # Macierz z zredukowanymi wierszami
    reduced_matrix: np.ndarray = np.zeros(matrix.shape)  # Zredukowana macierz

   # Redukcja wierszy
    for i in range(matrix.shape[0]):
        if np.array_equal(matrix[i], matrix.shape[0]*[np.inf]) is False:
            minimum = matrix[i].min()
            reduction_cost += minimum
            row_reduced = [x - minimum for x in matrix[i]]
            reduced_rows_matrix[i] = row_reduced
        else:
            reduced_rows_matrix[i] = matrix[i]

    print(f'Macierz ze zredukowanymi wierszami:\n {reduced_rows_matrix}')
    print(f'Koszt redukcji wierszy: {reduction_cost}')

    # Redukcja kolumn
    for i in range(matrix.shape[0]):
        if np.array_equal(matrix[:, i], matrix.shape[0]*[np.inf]) is False:
            minimum = reduced_rows_matrix[:, i].min()
            reduction_cost += minimum
            col_reduced = [x - minimum for x in reduced_rows_matrix[:, i]]
            reduced_matrix[:, i] = col_reduced
        else:
            reduced_matrix[:, i] = matrix[:, i]

    print(f'Zredukowana macierz:\n {reduced_matrix}')
    print(f'Całkowity koszt redukcji macierzy: {reduction_cost}')

    return reduced_matrix, reduction_cost


def get_cost(row: int, col: int, A: np.ndarray) -> int:
    '''
    Funkcja obliczająca koszt wyłączenia odcinka o danych współrzędnych (row, col).
    Koszt to suma najmniejszego elementu w kolumnie col oraz najmniejszego elementu w wierszu row, nie               \
    uwzględniając elementu (row, col)

    :param row: numer wiersza
    :param col: numer kolumny
    :param A: macierz
    :return: koszt wyłączenia odcinka o danych współrzędnych
    '''

    X, Y = A.shape
    row_without_zero = []
    col_without_zero = []

    for i in range(Y):
        if i != col:
            row_without_zero.append(A[row][i])

    for i in range(X):
        if i != row:
            col_without_zero.append(A[i][col])

    cost = np.min(np.array(row_without_zero)) + np.min(np.array(col_without_zero))

    return cost


## Krok 2 - Wyznaczenie odcinka o maksymalnym otymistycznym koszcie wyłączenia

In [3]:
def get_new_coords(A: np.ndarray) -> Tuple[Tuple[int, int], int]:
    '''
    Funkcja wyznaczająca współrzędne nowego odcinka
    PRZYJĘTE OZNACZENIE PRZEJŚĆ ZABRONIONYCH JAKO NP.INF

    :param A: Zredukowna macierz kosztów
    :return: dwuelementowa krotka zawierająca współrzędne początka oraz końca odcinka <i*j*>,
             max optymalny koszt wyłączenia tego odcinka
    '''
    
    print("\nKrok 2\n")
    
    X, Y = A.shape
    cost: int = -1

    #  współrzędne nowego odcinka
    new_row: int = 0
    new_col: int = 0

    for i in range(X):
        for j in range(Y):
            # Szukam zer w zredukowanej macierzy
            if A[i][j] != np.inf:
                if A[i][j] == 0:
                    #  koszt wyłączenia - suma elementów minimalnych w wierszu 'i' oraz kolumnie 'j' bez                                rozważanego elementu zerowego
                    suma = get_cost(i, j, A)
                    #  Wybieramy te współrzędne, dla których koszt wyłączenia jest największy
                    if suma > cost:
                        new_row, new_col = i, j
                        cost = suma
                        
    print(f'Wyznaczony odcinek: {new_row, new_col}, koszt wyłączenia: {cost}')
    return (new_row, new_col), cost
    

## Krok 3 - Podział problemu na dwa podproblemy

In [4]:
def subproblems(data: np.ndarray, path: List[Tuple[int, int]], section: Tuple[int, int], LB: int):
    """
    funkcja przyjmuje jako argumenty:
    data: Macierz z danymi
    path: dotychczas wyznaczone odcinki
    section: nowo wyznaczony odcinek
    LB: dotychczasowe LB
    
    zwraca:
    red_first_sub - zredukowany pierwszy podproblem zawierający odcinek <i*j*>
    new_LB - nowo wyznaczone LB dla red_first_sub
    red_second_sub - zredukowany drugi podproblem nie zawierający odcinka <i*j*>
    new_LB2 - nowo wyznaczone LB dla red_second_sub
    """
    
    print("\nKrok 3\n")
    
    # PODPROBLEM PIERWSZY
    first_sub: np.ndarray = data.copy()
    
    # Wykreślenie i-tego wiersza oraz j-tej kolumny
    for i in range(first_sub.shape[0]):
        if i == section[0]:
            first_sub[i] = [np.inf] * len(first_sub[i])
        else:
            first_sub[i][section[1]] = np.inf
        
    # zabronienie podcyklu
    path.append(section)
    subcircles(first_sub, path)
    
    # dodatkowa redukcja i nowe LB
    red_first_sub, cost = reduce_matrix(first_sub)
    new_LB = LB + cost
    
    print("Macierz zredukowana:\n", red_first_sub, f"\nNowe LB = {LB} + {cost} = {new_LB}")
    
    
    # PODPROBLEM DRUGI
    # zabronienie <i*j*>
    second_sub: np.ndarray = data.copy()
    
    # redukcja 
    red_second_sub, cost = reduce_matrix(second_sub)
    
    # LB
    new_LB2 = LB + cost
    
    print("Macierz zredukowana:\n", red_second_sub, f"\nNowe LB = {LB} + {cost} = {new_LB2}")
    
    return [(red_first_sub, new_LB), (red_second_sub, new_LB2)]
    

In [5]:
def subcircles(data: np.ndarray, section: List[Tuple[int, int]]) -> None:
    """
    data: macierz z danymi
    section: dotychczas wyznaczone odcinki
    Funkcja zabrania powstawania podcykli
    """
    if len(section) < data.shape[0]:
        p = section[-1][0]
        k = section[-1][1]
        begins = [elem[0] for elem in section]
        ends = [elem[1] for elem in section]

        # znalezienie początku i końca odcinka
        while True:
            if p in ends:
                p = section[ends.index(p)][0]
            else:
                break
        
        while True:
            if k in begins:
                k = section[begins.index(k)][1]
            else:
                break
        
        # zabronienie podcyklu
        data[k][p] = np.inf
        
        
        

## Krok 4 - Analiza podproblemów i kryterium zakończenia obliczeń

## Krok 5 - Wybór podproblemu o minimalnej wartości kosztu redukcji

# Algorytm Little'a

In [6]:
def Little_algorithm (matrix: np.ndarray):
    
    print("Macierz wejściowa:\n", matrix, '\n')
    
    path = []
    list_of_subproblems = []
    stop = False
    
    reduced_matrix, reduction_cost = reduce_matrix(matrix) #Krok 1
    
    #while stop == False
    
    new_vertex, LB = get_new_coords(reduced_matrix) #Krok 2
    path.append(new_vertex)
    
    list_of_subproblems = subproblems(reduced_matrix, path, new_vertex, LB) #Krok 3
    
    return path

matrix = np.array([[5, 8, 9, 2, 1, 5, 8, 6, 4, 2],
                   [3, 4, 5, 8, 7, 5, 2, 1, 6, 5],
                   [9, 8, 7, 5, 9, 2, 3, 6, 5, 9],
                   [8, 7, 5, 6, 5, 7, 8, 9, 5, 6],
                   [2, 1, 4, 5, 7, 8, 6, 3, 2, 1],
                   [4, 7, 5, 8, 9, 6, 5, 2, 1, 4],
                   [7, 9, 5, 6, 3, 2, 1, 4, 5, 4],
                   [8, 2, 1, 5, 3, 2, 1, 2, 3, 6],
                   [5, 7, 5, 6, 2, 7, 9, 3, 4, 6],
                   [7, 5, 6, 4, 6, 7, 9, 2, 1, 7]])

Little_algorithm(matrix)

Macierz wejściowa:
 [[5 8 9 2 1 5 8 6 4 2]
 [3 4 5 8 7 5 2 1 6 5]
 [9 8 7 5 9 2 3 6 5 9]
 [8 7 5 6 5 7 8 9 5 6]
 [2 1 4 5 7 8 6 3 2 1]
 [4 7 5 8 9 6 5 2 1 4]
 [7 9 5 6 3 2 1 4 5 4]
 [8 2 1 5 3 2 1 2 3 6]
 [5 7 5 6 2 7 9 3 4 6]
 [7 5 6 4 6 7 9 2 1 7]] 

Krok 1

Macierz ze zredukowanymi wierszami:
 [[4. 7. 8. 1. 0. 4. 7. 5. 3. 1.]
 [2. 3. 4. 7. 6. 4. 1. 0. 5. 4.]
 [7. 6. 5. 3. 7. 0. 1. 4. 3. 7.]
 [3. 2. 0. 1. 0. 2. 3. 4. 0. 1.]
 [1. 0. 3. 4. 6. 7. 5. 2. 1. 0.]
 [3. 6. 4. 7. 8. 5. 4. 1. 0. 3.]
 [6. 8. 4. 5. 2. 1. 0. 3. 4. 3.]
 [7. 1. 0. 4. 2. 1. 0. 1. 2. 5.]
 [3. 5. 3. 4. 0. 5. 7. 1. 2. 4.]
 [6. 4. 5. 3. 5. 6. 8. 1. 0. 6.]]
Koszt redukcji wierszy: 16
Zredukowana macierz:
 [[3. 7. 8. 0. 0. 4. 7. 5. 3. 1.]
 [1. 3. 4. 6. 6. 4. 1. 0. 5. 4.]
 [6. 6. 5. 2. 7. 0. 1. 4. 3. 7.]
 [2. 2. 0. 0. 0. 2. 3. 4. 0. 1.]
 [0. 0. 3. 3. 6. 7. 5. 2. 1. 0.]
 [2. 6. 4. 6. 8. 5. 4. 1. 0. 3.]
 [5. 8. 4. 4. 2. 1. 0. 3. 4. 3.]
 [6. 1. 0. 3. 2. 1. 0. 1. 2. 5.]
 [2. 5. 3. 3. 0. 5. 7. 1. 2. 4.]
 [5. 4. 5. 2. 5. 6. 8. 1.

[(1, 7), (1, 7)]